In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("category_dataset.csv", index_col = False)
df.head()

,Date & Time,Source IP,Destination IP,Source MAC,Destination MAC,Protocol,Flags,Src Port,Dst Port,Length,Class
0,27-02-2020 11:34,192.168.0.50,192.168.0.20,40:f2:e9:9d:44:6b,00:90:e8:5e:d1:66,TCP,AP,60394,502,64,Normal
1,27-02-2020 11:34,192.168.0.20,192.168.0.50,00:90:e8:5e:d1:66,40:f2:e9:9d:44:6b,TCP,A,502,60394,40,Normal
2,27-02-2020 11:34,192.168.0.50,192.168.0.20,40:f2:e9:9d:44:6b,00:90:e8:5e:d1:66,TCP,AP,60394,502,52,Normal
3,27-02-2020 11:34,192.168.0.21,192.168.0.5,20:47:47:87:d4:96,00:1d:9c:a1:89:a4,TCP,S,59170,20000,48,Normal
4,27-02-2020 11:34,192.168.0.21,192.168.0.5,20:47:47:87:d4:96,00:1d:9c:a1:89:a4,TCP,A,59170,20000,40,Normal


In [3]:
def change_class(data):
    lst = ['TCP port & service discovery','Stealth scan OS & version detection','Malware']
    if data in lst:
        return "Malware stealth scan"
    else:
        return data

In [4]:
df["Class"] = df["Class"].apply(change_class)

In [7]:
from joblib import dump, load
dnn_enc_bin = load('dnn_bin_enc_pca.joblib')
dnn_sc_enc = load('dnn_sc_enc_pca.joblib')
dnn_le = load('dnn_le_pca.joblib')
dnn_pca = load('dnn_pca_enc.joblib')

In [8]:
from keras.models import load_model
dnn_model = load_model("dnn_multiclass_pca.h5")

Using TensorFlow backend.


In [15]:
from datetime import datetime
start = []
stop = []
true_class = []
pred_class = []

In [16]:
for i in range(10000):
    test = df.sample(n=1)
    true_class.append(*list(test["Class"].values))
    test = test[['Source IP', 'Destination IP', 'Protocol', 'Flags', 'Src Port', 'Dst Port','Length']].values
    start_time = datetime.now()
    test_bin = dnn_enc_bin.transform(test)
    test_scale = dnn_sc_enc.transform(test_bin)
    test_pca = dnn_pca.transform(test_scale)
    prediction_test = dnn_model.predict_classes(test_pca)
    stop_time = datetime.now()
    start.append(start_time)
    stop.append(stop_time)
    pred_class.extend(dnn_le.inverse_transform(prediction_test))

In [19]:
def find_delay(timestamp):
    start_ms = timestamp[0].microsecond
    stop_ms = timestamp[1].microsecond
    delay = stop_ms - start_ms
    if delay >= 0:
        return delay
    else:
        return 1000000+delay

In [20]:
test_dnn_df = pd.DataFrame({"Start time":start, "Stop time":stop, "True Class":true_class, "Predicted Class":pred_class})
test_dnn_df["Delay"] = test_dnn_df[["Start time","Stop time"]].apply(find_delay,axis=1)
test_dnn_df.head()

,Start time,Stop time,True Class,Predicted Class,Delay
0,2020-05-25 23:18:28.200427,2020-05-25 23:18:28.213585,Ping scan,Ping scan,13158
1,2020-05-25 23:18:28.213585,2020-05-25 23:18:28.229208,DOS UDP scan,Malware stealth scan,15623
2,2020-05-25 23:18:28.229208,2020-05-25 23:18:28.229208,Malware stealth scan,Malware stealth scan,0
3,2020-05-25 23:18:28.244828,2020-05-25 23:18:28.260451,DOS UDP scan,Malware stealth scan,15623
4,2020-05-25 23:18:28.260451,2020-05-25 23:18:28.276073,UDP port & service scan,UDP port & service scan,15622


In [21]:
test_dnn_df["Delay"].describe()

count    10000.000000
mean     11348.122400
std       7120.693653
min          0.000000
25%          0.000000
50%      15621.000000
75%      15622.000000
max      32872.000000
Name: Delay, dtype: float64

In [ ]:
test_dnn_df.to_csv("dnn_dealy_pca.csv", index=False)